# Linear Algebra

In [1]:
import iarray as ia
import numpy as np

## Matrix multiplication

Matrix multiplication is a binary operation that produces a matrix from two matrices.
Computing matrix products is a central operation in all computational applications of
linear algebra.

In this part of the tutorial we will see how matrix multiplication works in *ironArray*.

### Matrix-Matrix multiplication

First, we are going to create two chunked, compressed matrices using *ironArray*.

In [2]:
chunkshape=(1000, 1000)
blockshape=(250, 250)

adtshape = ia.DTShape((3000, 5000), dtype=np.float64)
a = ia.linspace(adtshape, 0, 1, chunkshape=chunkshape, blockshape=blockshape)

bdtshape = ia.DTShape((5000, 4000), dtype=np.float64)
b = ia.linspace(bdtshape, 0, 1, chunkshape=chunkshape, blockshape=blockshape)


Unlike numpy, *ironArray* performs the matrix-matrix multiplication using blocks. In this way, instead
of decompressing all the data from the two arrays, we only have to decompress the data needed in
each block operation (allowing larger operations with less memory).

If we use the operator `@` to permorm the multiplications in Iron Array, all the params of the output array will be those defined in the global configuration.

In [3]:
c = a @ b
c.info

type                : IArray
shape               : (3000, 4000)
chunkshape          : (2048, 2048)
blockshape          : (128, 256)


As the chunkshape and the blockshape are not defined in the global configuration, they are automatically defined in the output array.

### Matrix-Vector multiplication

*ironArray* also allows users to perform matrix-vector multiplication in the same way that
matrix-matrix multiplication is performed.

However, if we want to set some parameters in the out array (for example, the chunkshape and the blockshape), we must use the `ia.matmul` function to perform the multiplication.

In [4]:
adtshape = ia.DTShape((3000, 5000), dtype=np.float64)
a = ia.linspace(adtshape, -2, -1, chunkshape=(1000, 1000), blockshape=(250, 250))

bdtshape = ia.DTShape((5000,), dtype=np.float64)
b = ia.linspace(bdtshape, 5, 10, chunkshape=(1000,), blockshape=(250,))

c = ia.matmul(a, b, chunkshape=(1000,), blockshape=(200,))

c.info

type                : IArray
shape               : (3000,)
chunkshape          : (1000,)
blockshape          : (200,)


Now, the array output has the chunkshape and the blockshape that we have specified in the `ia.matmul`function.

### Transpose

In Iron Array, the transpose of an array is computed using views. It can be obtained using different methods/functions:

In [5]:
at0 = a.T
at1 = a.transpose()
at2 = ia.transpose(a)

at0.is_view(), at1.is_view(), at2.is_view()

(True, True, True)

If we do not want a view, we just copy the transposed array.

In [6]:
at = a.T.copy()

### Basic example:  Neural Networks

Finally, we are going to see a simple example of what we can do with the functions described above. We are going to implement an simple neural network using these functions and some other Iron Array modules like expressions or random generators.

First of all, we have to generate the dataset that we use in our NN.

In [7]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

coord, cl = make_classification(20000, 1000)
X, Xt, y, yt = train_test_split(coord, cl, test_size=0.3)

training_inputs = ia.numpy2iarray(X.astype(np.float64))
training_labels = ia.numpy2iarray(y.astype(np.float64).reshape(-1, 1))
inputs = ia.numpy2iarray(Xt.astype(np.float64))

Now, we create a class implementing a neural network with a hidden layer.

In [8]:
class NN(object):
    
    def __init__(self, input_layer, hidden_layer, output_layer, random_seed=1):
        ia.set_config(seed=random_seed)
        self.l1_weights = ia.irandom.standard_normal(ia.DTShape((input_layer, hidden_layer)))
        self.l2_weights = ia.irandom.standard_normal(ia.DTShape((hidden_layer, output_layer)))

    def _sigmoid(self, x):
        return ia.expr_from_string("1 / (1 + exp(-x))", {"x": x}).eval()

    def _sigmoid_prime(self, x):
        return ia.expr_from_string("x * (1 - x)", {"x": x}).eval()
    
    def feed_forward(self, X):
        l1_output = ia.matmul(X, self.l1_weights)
        l1_output = self._sigmoid(l1_output)

        l2_output = ia.matmul(l1_output, self.l2_weights)
        l2_output = self._sigmoid(l2_output)

        return l1_output, l2_output
    
    def backpropagation(self, l1, l2, y):
        
        l2_error = (y - l2).eval()
        
        l2_delta = (l2_error * self._sigmoid_prime(l2)).eval()

        l1_error = ia.matmul(l2_delta, self.l2_weights.T)

        l1_delta = (l1_error * self._sigmoid_prime(l1)).eval()
        
        return l2_error, l1_delta, l2_delta

    def update_weights(self, X, l1, l1_delta, l2_delta, alpha=0.01):
        expr = f"w + {alpha} * d"
        d2 = ia.matmul(l1.T, l2_delta)
        d1 = ia.matmul(X.T, l1_delta)

        self.l2_weights = ia.expr_from_string(expr, {"w": self.l2_weights, "d": d2}).eval()
        self.l1_weights = ia.expr_from_string(expr, {"w": self.l1_weights, "d": d1}).eval()

    def predict(self, X):
        _, l2 = self.feed_forward(X)
        return l2

    def train(self, X, y, threshold=400, alpha=0.01):
        for j in range(threshold + 1):
            l1, l2 = self.feed_forward(X)
            l2_error, l1_delta, l2_delta = self.backpropagation(l1, l2, y)
            self.update_weights(X, l1, l1_delta, l2_delta, alpha=alpha)
            if(j % 100 == 0):
                train_error = ia.mean(ia.abs(l2_error).eval())
                print("epoch {:5} ".format(j),end='-')
                print(' error: {:0.4f} '.format(train_error))

Once the NN is created, we create a NN and train it with our dataset.

In [ ]:
nn = NN(X.shape[1], 5, 1)

nn.train(training_inputs, training_labels, threshold=500, alpha=0.1)

epoch     0 - error: 0.5080 
epoch   100 - error: 0.1206 
epoch   200 - error: 0.1010 
epoch   300 - error: 0.0999 
epoch   400 - error: 0.0995 


Finally, we predict the test part of our dataset and print the accuracy score :)

In [ ]:
labels = nn.predict(inputs)
labels = np.ravel((labels.data > 0.5).astype(int))
accuracy_score(labels, yt)